In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

path_to_file = "corpus.txt"

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Length of text: 782819 characters
90 unique characters


In [2]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])



In [3]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [4]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [5]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [6]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [8]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

In [9]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text100'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [10]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_29'

In [11]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [12]:
optimizer = tf.train.AdamOptimizer()

In [13]:
# Training step
EPOCHS = 30

for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)
              
          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

          if batch_n % 100 == 0:
              template = 'Epoch {} Batch {} Loss {:.4f}'
              print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.4993
Epoch 1 Batch 100 Loss 2.1265
Epoch 1 Loss 1.9905
Time taken for 1 epoch 8.148773193359375 sec

Epoch 2 Batch 0 Loss 2.0355
Epoch 2 Batch 100 Loss 1.4726
Epoch 2 Loss 1.3557
Time taken for 1 epoch 6.432200193405151 sec

Epoch 3 Batch 0 Loss 1.3230
Epoch 3 Batch 100 Loss 1.1995
Epoch 3 Loss 1.1470
Time taken for 1 epoch 6.415666818618774 sec

Epoch 4 Batch 0 Loss 1.1649
Epoch 4 Batch 100 Loss 1.0452
Epoch 4 Loss 1.0083
Time taken for 1 epoch 6.448187351226807 sec

Epoch 5 Batch 0 Loss 0.9670
Epoch 5 Batch 100 Loss 0.9523
Epoch 5 Loss 0.9329
Time taken for 1 epoch 7.229085445404053 sec

Epoch 6 Batch 0 Loss 0.8667
Epoch 6 Batch 100 Loss 0.8636
Epoch 6 Loss 0.9522
Time taken for 1 epoch 6.648202180862427 sec

Epoch 7 Batch 0 Loss 0.8943
Epoch 7 Batch 100 Loss 0.8314
Epoch 7 Loss 0.9082
Time taken for 1 epoch 6.63148832321167 sec

Epoch 8 Batch 0 Loss 0.7998
Epoch 8 Batch 100 Loss 0.8326
Epoch 8 Loss 0.9265
Time taken for 1 epoch 6.301945209503174 sec

Epoch 9 B

In [14]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_29'

In [15]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23040     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 90)             92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [17]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  #num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = .8
  # Here batch size == 1
  model.reset_states()
  test = []
  while test != '\n':#for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      test=idx2char[predicted_id]
      text_generated.append(test)
    
  return (start_string + ''.join(text_generated))

In [18]:
#change inputw to what you would like to draw
inputw='Draw a line'

output_instruction = ''
output_instruction+=(generate_text(model, "1. "  + inputw))
for i in range(2,11):
   output_instruction=(generate_text(model, output_instruction+str(i)+'.'))
#

In [19]:
print(output_instruction)

1. Draw a line and banner to the opposite side, forming a mirror image of the first line, fully enclosing the star and the stem. Atate the spiral shaped lines of the original rectangle.
2. ​​​​​​Draw a second egg shape from which it his completes the torso of a fun.
3. ​​​​​​​​​​​​​​Draw the ears. Use a curved line to connect the far side of the left line of the ears on purpose to look between the skin striped in the skeleton and bears, oo rode lines of various sizes with rounded corners.
4. Outline the leaves, enclose a curved shape at its base – these flames using short lines that meet in jagged points.
5. Draw the school bus. Most small squares and circles should be partially ed their the bright colors.
6. Draw many curved lines on the opposite side of the bat’s barrel and the sides of the figure to outline the donkey’s muzzle.
7. ​​​Draw a straight, horizontal line from the middle of each to meet in a point. Draw curved lines on the opposite side, forming the opposite side of the p